In [1]:
import pandas as pd
import sqlite3
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
public = pd.read_csv('./buildings_dev_pha_clean_1.0.csv')

In [3]:
cols_to_front = ['corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5']
remaining_cols = [col for col in public.columns if col not in cols_to_front]
public = public[cols_to_front + remaining_cols]

In [4]:
public = public[public.corrected_address.notna()]

In [5]:
public.std_zip5 = public.std_zip5.astype(str)
public.std_zip5 = public.std_zip5.replace('nan', '')
public.std_zip5 = public.std_zip5.apply(lambda x: x.split('.')[0])
public.std_zip5 = public.std_zip5.apply(lambda x: '00' + x if len(x) == 3 else x)
public.std_zip5 = public.std_zip5.apply(lambda x: '0' + x if len(x) == 4 else x)

In [6]:
public['corrected_address'] = public.corrected_address.fillna('')
public['corrected_address'] = public['corrected_address'].str.replace(r'\s+', ' ').str.strip()

public['corrected_unit'] = public.corrected_unit.fillna('')
public['corrected_unit'] = public.corrected_unit.str.strip()

public['std_city'] = public.std_city.fillna('')
public['std_city'] = public.std_city.str.strip()

public['std_st'] = public.std_st.fillna('')
public['std_st'] = public.std_st.str.strip()

C:\Users\jackv\AppData\Local\Temp\ipykernel_12628\3736073915.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  public['corrected_address'] = public['corrected_address'].str.replace(r'\s+', ' ').str.strip()


In [7]:
full_fema_matches = pd.DataFrame()

In [8]:
for year in range(2012, 2022):
    conn = sqlite3.Connection('./data/nfirs/fire_data.db')
    df = pd.read_sql("""
        select 
            INCIDENT_KEY,
            address,
            APT_NO,
            CITY,
            STATE,
            ZIP5,
            PROP_LOSS,
            CONT_LOSS,
            OTH_DEATH,
            OTH_INJ,
            INC_TYPE,
            HUM_FAC_1,
            HUM_FAC_2,
            HUM_FAC_3,
            HUM_FAC_4,
            HUM_FAC_5,
            HUM_FAC_6,
            HUM_FAC_7,
            BLDG_INVOL,
            DET_ALERT,
            FLAME_SPRD,
            AES_PRES,
            ST_DAM_HVY,
            ST_DAM_SIG,
            ST_DAM_XTR,
            ST_DAM_MIN 
        from buildings_fires_all 
        where INC_YEAR
        """ + f" = {year};", conn)

    df['EXP_NO'] = df.INCIDENT_KEY.apply(lambda x: x.split('_')[-1]).astype(int)

    df['address'] = df.address.fillna('')
    df['address'] = df['address'].str.replace(r'\s+', ' ').str.strip()

    df['APT_NO'] = df.APT_NO.fillna('')
    df['APT_NO'] = df.APT_NO.str.strip()

    df['CITY'] = df.CITY.fillna('')
    df['CITY'] = df.CITY.str.strip()

    df['STATE'] = df.STATE.fillna('')
    df['STATE'] = df.STATE.str.strip()

    df.ZIP5 = df.ZIP5.astype(str)
    df.ZIP5 = df.ZIP5.replace('nan', '')
    df.ZIP5 = df.ZIP5.apply(lambda x: x.split('.')[0])
    df.ZIP5 = df.ZIP5.apply(lambda x: '00' + x if len(x) == 3 else x)
    df.ZIP5 = df.ZIP5.apply(lambda x: '0' + x if len(x) == 4 else x)

    df['HUM_FAC_1'] = df.HUM_FAC_1.apply(lambda x: 1 if x == '1' else 0)
    df['HUM_FAC_2'] = df.HUM_FAC_2.apply(lambda x: 1 if pd.notna(x) else 0)
    df['HUM_FAC_3'] = df.HUM_FAC_3.apply(lambda x: 1 if pd.notna(x) else 0)
    df['HUM_FAC_4'] = df.HUM_FAC_4.apply(lambda x: 1 if pd.notna(x) else 0)
    df['HUM_FAC_5'] = df.HUM_FAC_5.apply(lambda x: 1 if pd.notna(x) else 0)
    df['HUM_FAC_6'] = df.HUM_FAC_6.apply(lambda x: 1 if pd.notna(x) else 0)
    df['HUM_FAC_7'] = df.HUM_FAC_7.apply(lambda x: 1 if pd.notna(x) else 0)
    df['BLDG_INVOL'] = df.BLDG_INVOL.apply(lambda x: np.nan if pd.isna(x) or float(x) < 0 else x)
    df['BLDG_INVOL'] = df.BLDG_INVOL.astype(float)
    df['DET_ALERT'] = df.DET_ALERT.replace('U', np.nan)
    df['DET_ALERT'] = df.DET_ALERT.apply(lambda x: 1 if x == '1' else 0)
    df['FLAME_SPRD'] = df.FLAME_SPRD.apply(lambda x: 1 if x == 'Y' else 0)
    df['AES_PRES'] = df.AES_PRES.replace('U', np.nan)
    df['AES_PRES'] = df.AES_PRES.replace('N', 0)
    df['AES_PRES'] = df.AES_PRES.apply(lambda x: 1 if x == '1' else .5)
    df['ST_DAM_MIN'] = df.ST_DAM_MIN.apply(lambda x: np.nan if pd.isna(x) or float(x) < 0 else x)
    df['ST_DAM_XTR'] = df.ST_DAM_XTR.apply(lambda x: np.nan if pd.isna(x) or float(x) < 0 else x)
    df['ST_DAM_HVY'] = df.ST_DAM_HVY.apply(lambda x: np.nan if pd.isna(x) or float(x) < 0 else x)
    df['ST_DAM_SIG'] = df.ST_DAM_SIG.apply(lambda x: np.nan if pd.isna(x) or float(x) < 0 else x)

    col = 'INC_TYPE'
    dummies = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df.drop(col, axis=1), dummies], axis=1)

    inc_type_cols = [x for x in df.columns if 'INC_TYPE_' in x]
    df['building_fires'] = df[inc_type_cols].sum(axis=1)

    df = df.drop(['INC_TYPE_114', 'INC_TYPE_115', 'INC_TYPE_116'], axis=1)

    full_fema_matches = pd.concat([full_fema_matches, public.merge(df, how='inner', 
        left_on=['corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5'],
        right_on=['address', 'APT_NO', 'CITY', 'STATE', 'ZIP5']
    )], axis=0)

    # Max for indicator variable columns
    aggregated = df \
        .drop(['INCIDENT_KEY'], axis=1) \
        .groupby(['address', 'APT_NO', 'CITY', 'STATE', 'ZIP5']).agg({
            'PROP_LOSS': 'sum',
            'CONT_LOSS': 'sum',
            'OTH_DEATH': 'sum',
            'OTH_INJ': 'sum',
            'HUM_FAC_1': 'max',
            'HUM_FAC_2': 'max',
            'HUM_FAC_3': 'max',
            'HUM_FAC_4': 'max',
            'HUM_FAC_5': 'max',
            'HUM_FAC_6': 'max',
            'HUM_FAC_7': 'max',
            'BLDG_INVOL': 'sum',
            'DET_ALERT': 'max',
            'FLAME_SPRD': 'max',
            'AES_PRES': 'max',
            'ST_DAM_HVY': 'sum',
            'ST_DAM_SIG': 'sum',
            'ST_DAM_XTR': 'sum',
            'ST_DAM_MIN': 'sum',
            'EXP_NO': 'max',
            'INC_TYPE_111': 'sum',
            'INC_TYPE_113': 'sum',
            'INC_TYPE_118': 'sum',
            'building_fires': 'sum'
    }).reset_index()

    new_cols = aggregated.columns

    df = df.sample(frac=1).reset_index(drop=True)

    public = public.merge(aggregated, how='left', 
        left_on=['corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5'],
        right_on=['address', 'APT_NO', 'CITY', 'STATE', 'ZIP5']
    )

    duplicate_indices_to_zero_out = public[
        (public.duplicated(subset=['corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5'], keep='first')) &
        (public.building_fires > 0)
    ].index

    public.iloc[duplicate_indices_to_zero_out] = 0

    public = public.drop(['address', 'APT_NO', 'CITY', 'STATE', 'ZIP5'], axis=1)
    fill_nas = [
        'PROP_LOSS', 'CONT_LOSS', 'OTH_DEATH', 'OTH_INJ', 'HUM_FAC_1', 'HUM_FAC_2',
        'HUM_FAC_3', 'HUM_FAC_4', 'HUM_FAC_5', 'HUM_FAC_6', 'HUM_FAC_7', 'BLDG_INVOL',
        'DET_ALERT', 'FLAME_SPRD', 'AES_PRES', 'ST_DAM_HVY', 'ST_DAM_SIG', 'ST_DAM_XTR',
        'ST_DAM_MIN', 'EXP_NO', 'INC_TYPE_111', 'INC_TYPE_113', 'INC_TYPE_118', 'building_fires'
    ]
    public[fill_nas] = public[fill_nas].fillna(0)
    public.columns = [x + f'_{year}' if x in new_cols else x for x in public.columns]


C:\Users\jackv\AppData\Local\Temp\ipykernel_12628\2905111608.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address'] = df['address'].str.replace(r'\s+', ' ').str.strip()
C:\Users\jackv\AppData\Local\Temp\ipykernel_12628\2905111608.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address'] = df['address'].str.replace(r'\s+', ' ').str.strip()
C:\Users\jackv\AppData\Local\Temp\ipykernel_12628\2905111608.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address'] = df['address'].str.replace(r'\s+', ' ').str.strip()
C:\Users\jackv\AppData\Local\Temp\ipykernel_12628\2905111608.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  df['address'] = df['address'].str.replace(r'\s+', ' ').str.strip()
C:\Users\jackv\AppData\Local\Temp\ipykernel_12628\2905111608.py:38: FutureWa

In [9]:
public.columns = [x.lower() for x in public.columns]

In [10]:
full_fema_matches \
    .loc[:, ['corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5', 'INCIDENT_KEY']] \
    .to_csv('full_fema_matches.csv', index=False)

In [11]:
public.to_csv('ph_fires_matched_1.1.csv', index=False)